In [6]:
import os
import torch
from copy import deepcopy
import numpy as np
import xarray as xr
import pandas as pd
import torch.nn as nn
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [7]:
#设置种子
def set_seed(seed = 427):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)

In [8]:
#数据导入
def load_data2():
    # CMIP data    
    train = xr.open_dataset('../tcdata/enso_round1_train_20210201/CMIP_train.nc')
    label = xr.open_dataset('../tcdata/enso_round1_train_20210201/CMIP_label.nc')    
   
    train_sst = train['sst'][:, :12].values  # (4645, 12, 24, 72)截取前12项
    train_sst= np.concatenate((train_sst[:151*5],train_sst[151*9:151*12],train_sst[151*13:]))  
    train_t300 = train['t300'][:, :12].values
    train_t300= np.concatenate((train_t300[:151*5],train_t300[151*9:151*12],train_t300[151*13:]))  
    train_ua = train['ua'][:, :12].values
    train_ua= np.concatenate((train_ua[:151*5],train_ua[151*9:151*12],train_ua[151*13:]))  
    train_va = train['va'][:, :12].values
    train_va= np.concatenate((train_va[:151*5],train_va[151*9:151*12],train_va[151*13:]))  
    train_label = label['nino'][:, 12:36].values
    train_label= np.concatenate((train_label[:151*5],train_label[151*9:151*12],train_label[151*13:]))  
    
    #train_ua = np.nan_to_num(train_ua)#缺失值补0
    #train_va = np.nan_to_num(train_va)
    #train_t300 = np.nan_to_num(train_t300)
    #train_sst = np.nan_to_num(train_sst)
    # SODA data    
    train2 = xr.open_dataset('../tcdata/enso_round1_train_20210201/SODA_train.nc')
    label2 = xr.open_dataset('../tcdata/enso_round1_train_20210201/SODA_label.nc')
    
    train_sst2 = train2['sst'][:, :12].values  # (3890, 12, 24, 72)
    train_t3002 = train2['t300'][:, :12].values
    train_ua2 = train2['ua'][:, :12].values
    train_va2 = train2['va'][:, :12].values
    train_label2 = label2['nino'][:, 12:36].values

    print('Train samples: {}, Valid samples: {}'.format(len(train_label), len(train_label2)))

    dict_train = {
        'sst':train_sst,
        't300':train_t300,
        'ua':train_ua,
        'va': train_va,
        'label': train_label}
    dict_valid = {
        'sst':train_sst2,
        't300':train_t3002,
        'ua':train_ua2,
        'va': train_va2,
        'label': train_label2}
    train_dataset = EarthDataSet(dict_train)
    valid_dataset = EarthDataSet(dict_valid)
    return train_dataset, valid_dataset

class EarthDataSet(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data['sst'])

    def __getitem__(self, idx):   
        return (self.data['sst'][idx], self.data['t300'][idx], self.data['ua'][idx], self.data['va'][idx]), self.data['label'][idx]

In [9]:
class simpleSpatailTimeNN(nn.Module):
    def __init__(self, n_cnn_layer:int=1, kernals:list=[3,3], n_lstm_units:int=64):
        super(simpleSpatailTimeNN, self).__init__()
        self.conv1 = nn.ModuleList([nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3),
                                    nn.ReLU(inplace = True),
                                    nn.AvgPool2d(kernel_size=(2,2)),
                                    nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3),
                                    nn.ReLU(inplace = True)]) 
        self.conv2 = nn.ModuleList([nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3),
                                    nn.ReLU(inplace = True),
                                    nn.AvgPool2d(kernel_size=(2,2)),
                                    nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3),
                                    nn.ReLU(inplace = True)])
        self.conv3 = nn.ModuleList([nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3),
                                    nn.ReLU(inplace = True),
                                    nn.AvgPool2d(kernel_size=(2,2)),
                                    nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3),
                                    nn.ReLU(inplace = True)])
        self.conv4 = nn.ModuleList([nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3),
                                    nn.ReLU(inplace = True),
                                    nn.AvgPool2d(kernel_size=(2,2)),
                                    nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3),
                                    nn.ReLU(inplace = True)])
        self.pool1 = nn.AdaptiveAvgPool2d((22, 1))
        self.pool2 = nn.AdaptiveAvgPool2d((1, 70))
        self.pool3 = nn.AdaptiveAvgPool2d((1, 128))
        self.batch_norm = nn.BatchNorm1d(12, affine=False)
        self.lstm = nn.LSTM(9*33*4, n_lstm_units, 2, bidirectional=True)
        self.linear = nn.Linear(128, 24)

    def forward(self, sst, t300, ua, va):
        for conv1 in self.conv1:
            sst = conv1(sst)  # batch * 12 * (24 - 2-2) * (72 -2-2)
        for conv2 in self.conv2:
            t300 = conv2(t300)
        for conv3 in self.conv3:
            ua = conv3(ua)
        for conv4 in self.conv4:
            va = conv4(va)

        sst = torch.flatten(sst, start_dim=2)  # batch * 12 * 1540
        t300 = torch.flatten(t300, start_dim=2)
        ua = torch.flatten(ua, start_dim=2)
        va = torch.flatten(va, start_dim=2)  # if flat, lstm input_dims = 1540 * 4              
            
        x = torch.cat([sst, t300, ua, va], dim=-1) #在内层合并 batch*12*1540*4
        x = self.batch_norm(x)
        x, _ = self.lstm(x)#输入1540*4,64hidden,2layer且双向 输出batch*(128=64*2)
        x = self.pool3(x).squeeze(dim=-2)
        x = self.linear(x)#128-->24
        return x


#模型

class simpleSpatailTimeNN(nn.Module):
    def __init__(self, n_cnn_layer:int=1,n_lstm_units:int=64):
        super(simpleSpatailTimeNN, self).__init__()
        self.conv1 = nn.ModuleList([nn.Conv2d(in_channels=12, out_channels=12,kernel_size=3),#22*70
                                    nn.ReLU(inplace = True),
                                    nn.AvgPool2d(kernel_size=(2,2)),#11*35
                                    nn.Conv2d(in_channels=12, out_channels=12,kernel_size=3),#9*33
                                    nn.ReLU(inplace = True),
                                    nn.AvgPool2d(kernel_size=(3,3))] ),#3*11 
        self.conv2 = nn.ModuleList([nn.Conv2d(in_channels=12, out_channels=12,kernel_size=3),#22*70
                                    nn.ReLU(inplace = True),
                                    nn.AvgPool2d(kernel_size=(2,2)),#11*35
                                    nn.Conv2d(in_channels=12, out_channels=12,kernel_size=3),#9*33
                                    nn.ReLU(inplace = True),
                                    nn.AvgPool2d(kernel_size=(3,3))] ),#3*11 
        self.conv4 = nn.ModuleList([nn.Conv2d(in_channels=12, out_channels=12,kernel_size=3),#22*70
                                    nn.ReLU(inplace = True),
                                    nn.AvgPool2d(kernel_size=(2,2)),#11*35
                                    nn.Conv2d(in_channels=12, out_channels=12,kernel_size=3),#9*33
                                    nn.ReLU(inplace = True),
                                    nn.AvgPool2d(kernel_size=(3,3))] ),#3*11 
        self.pool1 = nn.AdaptiveAvgPool2d((22, 1))
        self.pool2 = nn.AdaptiveAvgPool2d((1, 70))
        self.pool3 = nn.AdaptiveAvgPool2d((1, 128))
        self.batch_norm = nn.BatchNorm1d(12, affine=False)
        self.lstm = nn.LSTM(33 * 4, n_lstm_units, 2, bidirectional=True)
        self.linear = nn.Linear(128, 24)

    def forward(self, sst, t300, ua, va):
        for conv1 in self.conv1:
            sst = conv1(sst)  # batch * 12 * (3*11)
        for conv2 in self.conv2:
            t300 = conv2(t300)
        for conv3 in self.conv3:
            ua = conv3(ua)
        for conv4 in self.conv4:
            va = conv4(va)

        sst = torch.flatten(sst, start_dim=2)  # batch *33
        t300 = torch.flatten(t300, start_dim=2)
        ua = torch.flatten(ua, start_dim=2)
        va = torch.flatten(va, start_dim=2)  # if flat, lstm input_dims = 1540 * 4              
            
        x = torch.cat([sst, t300, ua, va], dim=-1) #在内层合并 batch*12*33*4
        x = self.batch_norm(x)
        x, _ = self.lstm(x)#输入33*4,64hidden,2layer且双向 输出batch*(128=64*2)
        x = self.pool3(x).squeeze(dim=-2)
        x = self.linear(x)#128-->24
        return x


In [10]:
#loss function
def coreff(x, y):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    c1 = sum((x - x_mean) * (y - y_mean))
    c2 = sum((x - x_mean)**2) * sum((y - y_mean)**2)
    return c1/np.sqrt(c2)

def rmse(preds, y):
    return np.sqrt(sum((preds - y)**2)/preds.shape[0])

def eval_score(preds, label):
    # preds = preds.cpu().detach().numpy().squeeze()
    # label = label.cpu().detach().numpy().squeeze()
    acskill = 0
    RMSE = 0
    a = 0
    a = [1.5]*4 + [2]*7 + [3]*7 + [4]*6
    for i in range(24):
        RMSE += rmse(label[:, i], preds[:, i])
        cor = coreff(label[:, i], preds[:, i])
    
        acskill += a[i] * np.log(i+1) * cor
    return 2/3 * acskill - RMSE

In [11]:
#train
fit_params = {
    'n_epochs' : 50,
    'learning_rate' : 1e-4,
    'batch_size' : 128,
}


def train():
    #set_seed()
    train_dataset, valid_dataset = load_data2()      
    train_loader = DataLoader(train_dataset, batch_size=fit_params['batch_size'])
    valid_loader = DataLoader(valid_dataset, batch_size=fit_params['batch_size'])

    model = simpleSpatailTimeNN()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'   
    optimizer = torch.optim.Adam(model.parameters(), lr=fit_params['learning_rate'])
    loss_fn = nn.MSELoss()   
    
    model.to(device)
    loss_fn.to(device)

    for i in range(fit_params['n_epochs']):
        model.train()
        for step, ((sst, t300, ua, va), label) in enumerate(train_loader):                
            sst = sst.to(device).float()
            t300 = t300.to(device).float()
            ua = ua.to(device).float()
            va = va.to(device).float()
            optimizer.zero_grad()
            label = label.to(device).float()
            preds = model(sst, t300, ua, va)
            loss = loss_fn(preds, label)
            loss.backward()
            optimizer.step()
            #print('Step: {}, Train Loss: {}'.format(step, loss))

        model.eval()
        y_true, y_pred = [], []
        for step, ((sst, t300, ua, va), label) in enumerate(valid_loader):
            sst = sst.to(device).float()
            t300 = t300.to(device).float()
            ua = ua.to(device).float()
            va = va.to(device).float()
            label = label.to(device).float()
            preds = model(sst, t300, ua, va)
            y_pred.append(preds)
            y_true.append(label)

        y_true = torch.cat(y_true, axis=0)
        y_pred = torch.cat(y_pred, axis=0)
        sco = eval_score(y_true.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
        print('Epoch: {}, Valid Score {}'.format(i+1,sco))

        # torch.save(self.model.state_dict(), '../user_data/ref.pkl')
        #torch.save(model, '../notebook/score22_original')
        print('Model saved successfully')

In [ ]:
#删除缺失值
#新结构:先CNN卷四个分量kernel=5(20*68),然后avg pool 再卷积(stride=(5,4)-->4*17)再avg pool 再卷积(2*5),flatten再加月份 

In [12]:
train()
#details:1.用CMIP的值训练SODA,训练出前12个月[sst,t300,ua,va]预测后24个月nino的模型
#        2.结构:[sst,t300.ua,va]按照12个月分别过3*3卷积层(4channel,12*(24-2)*(72-2)),再flatten经纬度(4 features,12*1540)，最内层合并4个feature:(12*1540*4) 过batchnorm ,lstm：双向两层，输出batch*12*(64*2)  对12个月hidden平均池化，得到batch*128,全连接到24个月
#problems:1.缺失值填0 2.优化函数是MSE 3.没有利用前12个月的nino
#优化方向:需要突出空间经纬度相关性而不是直接flatten，更改网络结构，区分CMIP和SODA,ua va取滑动平均

Train samples: 3890, Valid samples: 100
Epoch: 1, Valid Score -14.052612775470948
Model saved successfully
Epoch: 2, Valid Score -6.453047160472959
Model saved successfully
Epoch: 3, Valid Score -1.521226632195738
Model saved successfully
Epoch: 4, Valid Score -2.0664745389544947
Model saved successfully
Epoch: 5, Valid Score 0.8222242320630535
Model saved successfully
Epoch: 6, Valid Score 9.637843065630417
Model saved successfully
Epoch: 7, Valid Score 21.527406711905222
Model saved successfully
Epoch: 8, Valid Score 31.458690347860685
Model saved successfully
Epoch: 9, Valid Score 39.27485391531059
Model saved successfully
Epoch: 10, Valid Score 45.74728350914828
Model saved successfully
Epoch: 11, Valid Score 51.032818361132364
Model saved successfully
Epoch: 12, Valid Score 55.31213904633023
Model saved successfully
Epoch: 13, Valid Score 59.0206992269091
Model saved successfully
Epoch: 14, Valid Score 62.42948315256115
Model saved successfully
Epoch: 15, Valid Score 65.4978036840